### A notebook evaluating a possible implementation for _live Boheh plots in a notebook cell_

In [ ]:
from __future__ import print_function
from collections import deque
from math import ceil, pi
import socket
import six
import ipywidgets as widgets
import numpy as np
from IPython.display import HTML
from tornado.ioloop import IOLoop
from bokeh.application import Application
from bokeh.application.handlers import Handler, FunctionHandler
from bokeh.embed import autoload_server
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import row, column, layout, gridplot
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.glyphs import Rect
from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.server.server import Server

In [ ]:
output_notebook()

In [ ]:
class SessionHandler(Handler):

    def on_server_loaded(self, server_context):
        print("SessionHandler.on_server_loaded called!")

    def on_server_unloaded(self, server_context):
        print("SessionHandler.on_server_unloaded called!")

    def on_session_created(self, session_context):
        print("SessionHandler.on_session_created called!")

    def on_session_destroyed(self, session_context):
        print("SessionHandler.on_session_destroye called!")

In [ ]:
class EmbeddedBokehServer(object):

    __bkh_app__ = None
    __bkh_srv__ = None
    __srv_url__ = None
    __clients__ = deque()

    @staticmethod
    def print_info():
        try:
            print("Bokeh server URL: {}".format(EmbeddedBokehServer.__srv_url__))
            sessions = EmbeddedBokehServer.__bkh_srv__.get_sessions()
            print("Number of opened sessions: {}".format(len(sessions)))
        except:
            pass
        
    @staticmethod
    def __start_server():
        app = Application(FunctionHandler(EmbeddedBokehServer.__entry_point))
        app.add(SessionHandler())
        srv = Server(
            {'/': app},
            io_loop=IOLoop.instance(),
            port=0,
            host='*',
            allow_websocket_origin=['*']
        )
        srv.start()
        srv_addr = srv.address if srv.address else socket.gethostbyname(socket.gethostname())
        EmbeddedBokehServer.__bkh_srv__ = srv
        EmbeddedBokehServer.__bkh_app__ = app
        EmbeddedBokehServer.__srv_url__ = 'http://{}:{}'.format(srv_addr, srv.port)
        
    @staticmethod
    def __entry_point(doc):
        try:
            #TODO: should we locked EmbeddedBokehServer.__callbacks__?
            client = EmbeddedBokehServer.__clients__.pop()
            model = client['mcb']()
            doc.add_root(model)
            doc.add_periodic_callback(client['pcb'], 1000. * client['prd'])
        except:
            pass

    @staticmethod
    def serve(model_cb, periodic_cb=None, period=1.0):
        if not EmbeddedBokehServer.__bkh_srv__:
            EmbeddedBokehServer.__start_server() 
        #TODO: should we locked EmbeddedBokehServer.__callbacks__?
        EmbeddedBokehServer.__clients__.appendleft({'mcb':model_cb, 'pcb':periodic_cb, 'prd':period})
        script = autoload_server(model=None, url=EmbeddedBokehServer.__srv_url__)
        html_display = HTML(script)
        display(html_display)

In [ ]:
# this is supposed to return a bokeh model (e.g. figure or layout)
def model_cb_1():
    p = figure(plot_width=850, plot_height=200)
    p.circle([1, 2, 3, 4, 5], [6, 8, 6, 4, 9], size=20, color="navy", alpha=0.5)
    return p

# periodic callback 
def periodic_cb_1():
    print("periodic_cb_1 called!")

In [ ]:
EmbeddedBokehServer.serve(model_cb_1, periodic_cb_1)

In [ ]:
# this is supposed to return a bokeh model (e.g. figure or layout)
def model_cb_2():
    p = figure(plot_width=850, plot_height=200)
    p.square([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], size=20, color="green", alpha=0.5)
    return p

# periodic callback 
def periodic_cb_2():
    print("periodic_cb_2 called!")

In [ ]:
EmbeddedBokehServer.serve(model_cb_2, periodic_cb_2)

In [ ]:
EmbeddedBokehServer.print_info()